# AWD LSTM $G_1$ & $G_2$ Classification with fast-ai

# Settings

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from IPython.display import display,HTML
from fastai.text.all import *
import pandas as pd

In [ ]:
path = r'/content'

# Fine tuning Encoder

In [ ]:
df_finetune = pd.read_csv(f'{path}\en_full.csv', low_memory=False)

In [ ]:
dls_finetune = TextDataLoaders.from_df(path=path, df=df_finetune, text_col='text', is_lm=True)
dls_finetune.show_batch(max_n=2)

In [ ]:
awd_finetune = language_model_learner(
    dls_finetune, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
awd_finetune.fit_one_cycle(5, 2e-2)

In [ ]:
awd_finetune.save_encoder('finetuned')

# Classification

In [ ]:
df_classif = pd.read_csv(f'{path}\en_labelled.csv', low_memory=False)

## AWD LSTM $G_1$

In [ ]:
dls_G1 = TextDataLoaders.from_df(path=path, df=df_classif, text_col='text', label_col='pnns_groups_1', is_lm=False, vocab=dls_finetune.vocab)
dls_G1.show_batch(max_n=2)

In [ ]:
AWD_G1 = text_classifier_learner(
    dls_G1, 
    AWD_LSTM(
        hidden_p=0.2, 
        input_p=0.6, 
        embed_p=0.1, 
        weight_p=0.5, 
        bidir=False
        ), 
    drop_mult=0.5, 
    metrics=PrecisionMulti
    ).to_fp16()

In [ ]:
AWD_G1 = AWD_G1.load_encoder('finetuned')

In [ ]:
AWD_G1.fit_one_cycle(1, 2e-2)

In [ ]:
AWD_G1.save('1epoch_G1')

In [ ]:
AWD_G1/load('1epoch_G1')

In [ ]:
AWD_G2.lr_find()

In [ ]:
AWD_G1.fit_one_cycle(20, 2e-2)

In [ ]:
interp_G1 = ClassificationInterpretation.from_learner(AWD_G1)

In [ ]:
interp_G1.print_classification_report()

In [ ]:
interp_G1.plot_confusion_matrix()

## AWD LSTM $G_2$

In [ ]:
dls_G2 = TextDataLoaders.from_df(path=path, df=df_classif, text_col='text', label_col='pnns_groups_2', is_lm=False, vocab=dls_finetune.vocab)
dls_G2.show_batch(max_n=2)

In [ ]:
AWD_G2 = text_classifier_learner(
    dls_G2, 
    AWD_LSTM(
        hidden_p=0.2, 
        input_p=0.6, 
        embed_p=0.1, 
        weight_p=0.5, 
        bidir=False
        ), 
    drop_mult=0.5, 
    metrics=PrecisionMulti
    ).to_fp16()

In [ ]:
AWD_G2 = AWD_G2.load_encoder('finetuned')

In [ ]:
AWD_G2.fit_one_cycle(1, 2e-2)

In [ ]:
AWD_G2.save('1epoch')

In [ ]:
AWD_G2.load('1epoch')

In [ ]:
AWD_G2.lr_find()

In [ ]:
learn.fit_one_cycle(20, 2e-2)

In [ ]:
interp_G2 = ClassificationInterpretation.from_learner(AWD_G2)

In [ ]:
interp_G2.print_classification_report()

In [ ]:
interp_G2.plot_confusion_matrix()

## Export models

In [ ]:
AWD_G1.save('awdg1')
AWD_G2.save('awdg2')
AWD_G1.export('awdg1_export')
AWD_G2.export('awdg2_export')